In [34]:
import pandas as pd
from tqdm.notebook import tqdm, trange
import re

storage = "/Volumes/easystore/Drones/"

cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [35]:
cv_flights['clean-type'] = cv_flights['type'].str.lower()
cv_flights['clean-type'] = cv_flights['clean-type'].apply(lambda x: str(x).replace('  ','').strip())

In [36]:
cv_flights[cv_flights['clean-type'].astype(str).str.contains('vaping')]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,clean-type


In [37]:
cv_flights['clean-type'].drop_duplicates().to_csv('../../data/manually-label/types-to-label-flights.csv',index=False)


In [38]:
cv_flights[cv_flights['clean-type'].str.contains('drone')]
cv_flights['src']='drone_flights'

In [39]:
calls_for_service['clean-type'] = calls_for_service['Description'].str.lower()
calls_for_service['src']='cfs'
calls_for_service['clean-type'] = calls_for_service['clean-type'].apply(lambda x: str(x).replace('  ','').strip())

In [40]:
a = []
a.append(calls_for_service[['src','clean-type']])
a.append(cv_flights[['src','clean-type']])
types = pd.concat(a)

In [41]:
types[['clean-type']].drop_duplicates()

,clean-type
0,extra patrol
1,narcotics
2,pedestrian stop
3,disturbance - general
5,pedestrian stop (person in parked veh)
...,...
9472,intoxicated person
9474,missing child / fight
9481,impaired driver / person with gun
9490,mental health call / mental health call


In [43]:
types[['clean-type']].drop_duplicates().to_csv('../../data/manually-label/types.csv',index=False)